In [96]:
# Importing libraries

#!pip install pandas
import pandas as pd
#!pip install numpy
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import re
warnings.filterwarnings("ignore")
%matplotlib inline

STEP 1 — Load all data sources

In [97]:
# Load all CSV files
amazon = pd.read_csv('/content/Amazon Sale Report.csv')
international = pd.read_csv('/content/International sale Report.csv')
may = pd.read_csv('/content/May-2022.csv')
sales = pd.read_csv('/content/Sale Report.csv')
expense = pd.read_csv('/content/Expense IIGF.csv')

print("Shapes:",
      "\nAmazon:", amazon.shape,
      "\nInternational:", international.shape,
      "\nMay:", may.shape,
      "\nSales:", sales.shape,
      "\nExpense:", expense.shape)


Shapes: 
Amazon: (128975, 24) 
International: (37432, 10) 
May: (1330, 17) 
Sales: (9271, 7) 
Expense: (17, 5)


STEP 2 — Normalize column names & build the unified sales dataset

In [98]:
def normalize_columns(df):
    out = df.copy()
    out.columns = out.columns.str.strip().str.lower().str.replace(' ', '_')
    return out

amazon = normalize_columns(amazon)
international = normalize_columns(international)
may = normalize_columns(may)
sales = normalize_columns(sales)

# Combine all sales data into one "flat file"
merged = pd.concat([amazon, international, may, sales], axis=0, ignore_index=True)
before = len(merged)
merged = merged.drop_duplicates()
print(f"Dropped {before - len(merged)} duplicate rows")
print("Unified sales dataset shape:", merged.shape)



Dropped 0 duplicate rows
Unified sales dataset shape: (177008, 47)


STEP 3 — Text cleaning, date conversion, and time features

In [99]:
def clean_text(x):
    if isinstance(x, str):
        x = x.lower()
        x = re.sub(r'[^a-z0-9 ]+', ' ', x)
        x = " ".join(x.split())
    return x

# Clean all text columns
text_cols = merged.select_dtypes(include='object').columns
for col in text_cols:
    merged[col] = merged[col].astype(str).apply(clean_text)

# Convert date columns to datetime
for col in merged.columns:
    if 'date' in col:
        merged[col] = pd.to_datetime(merged[col], errors='coerce')

# Create time-based features
if 'date' in merged.columns:
    merged['year'] = merged['date'].dt.year
    merged['month'] = merged['date'].dt.month
    merged['weekday'] = merged['date'].dt.weekday
    merged['is_weekend'] = merged['weekday'].isin([5,6]).astype(int)
else:
    print("⚠️ No main 'date' column detected – cannot derive year/month features.")

STEP 4 — Category simplification & missing value handling

In [100]:
# Reduce rare categories if 'category' exists
if 'category' in merged.columns:
    counts = merged['category'].value_counts()
    rare = counts[counts < 20].index
    merged['category'] = merged['category'].replace(rare, 'other')

# Safe imputation (no row deletion)
for col in merged.columns:
    if merged[col].isna().any():
        if merged[col].dtype == 'object':
            merged[col] = merged[col].fillna('unknown')
        else:
            merged[col] = merged[col].fillna(merged[col].median())

# Ensure quantity columns are numeric (useful for later ratios)
for q in ['qty','pcs']:
    if q in merged.columns:
        merged[q] = pd.to_numeric(merged[q], errors='coerce')


missing_percent = merged.isnull().mean() * 100
high_missing_cols = missing_percent[missing_percent > 80].index.tolist()

print("⚠️ Columns with >80% missing values:", len(high_missing_cols))
for c in high_missing_cols[:20]:
    print(f" - {c}: {missing_percent[c]:.2f}%")
print("Dataset shape (no deletion):", merged.shape)



⚠️ Columns with >80% missing values: 1
 - pcs: 100.00%
Dataset shape (no deletion): (177008, 51)


STEP 5 — Expense dataset: cleaning, monthly aggregation, and merge

In [101]:
# Optional: drop a stray 'index' column
if 'index' in expense.columns:
    expense = expense.drop(columns=['index'])

print("Expense headers BEFORE handling:", expense.columns.tolist())

# Detect if already clean
already_clean = {'raw_date','amount','description','note'}.issubset(expense.columns)

if not already_clean:
    # Try raw headers or normalized variants
    rename_map_candidates = [
        {'Recived Amount': 'raw_date', 'Unnamed: 1': 'amount', 'Expance': 'description', 'Unnamed: 3': 'note'},
        {'recived_amount': 'raw_date', 'unnamed:_1': 'amount', 'expance': 'description', 'unnamed:_3': 'note'}
    ]
    renamed = False
    for m in rename_map_candidates:
        if set(m.keys()).issubset(expense.columns):
            expense = expense.rename(columns=m); renamed = True; break
    if not renamed:
        # Fallback: normalize cols and try again
        expense = normalize_columns(expense)
        for m in rename_map_candidates:
            if set(m.keys()).issubset(expense.columns):
                expense = expense.rename(columns=m); renamed = True; break
        # If it was already clean post-normalization, this will pass:
        already_clean = {'raw_date','amount','description','note'}.issubset(expense.columns)
        if not (renamed or already_clean):
            raise ValueError("Could not map Expense headers to ['raw_date','amount','description','note'].")

print("Expense headers AFTER handling:", expense.columns.tolist())

# Ensure date/amount types; reuse existing date/year/month if present
if 'date' not in expense.columns:
    expense['date'] = pd.to_datetime(expense['raw_date'], errors='coerce')
else:
    expense['date'] = pd.to_datetime(expense['date'], errors='coerce')

expense['amount'] = pd.to_numeric(expense['amount'], errors='coerce')

# Keep valid rows
expense = expense[(expense['date'].notna()) & (expense['amount'].notna())].copy()

# Ensure year/month
if 'year' not in expense.columns:
    expense['year'] = expense['date'].dt.year
if 'month' not in expense.columns:
    expense['month'] = expense['date'].dt.month

# Aggregate monthly totals (idempotent)
monthly_expense = (
    expense.groupby(['year','month'], as_index=False)['amount']
           .sum()
           .rename(columns={'amount':'monthly_expense'})
)

print("Monthly expense preview:\n", monthly_expense.head())


Expense headers BEFORE handling: ['Recived Amount', 'Unnamed: 1', 'Expance', 'Unnamed: 3']
Expense headers AFTER handling: ['raw_date', 'amount', 'description', 'note']
Monthly expense preview:
    year  month  monthly_expense
0  2022      6           5000.0


STEP 6 — Merge expense into sales

In [102]:
# Guard: need year/month in sales
if not {'year','month'}.issubset(merged.columns):
    raise ValueError("Merged dataset lacks 'year'/'month'. Ensure STEP 3 created them from a 'date' column.")

merged = merged.merge(monthly_expense, on=['year','month'], how='left')
merged['monthly_expense'] = merged['monthly_expense'].fillna(0.0)

# expense per unit
if {'year','month'}.issubset(merged.columns):
    month_qty = merged.groupby(['year','month'])['qty' if 'qty' in merged.columns else 'pcs'].sum().rename('month_total_qty').reset_index()
    merged = merged.merge(month_qty, on=['year','month'], how='left')
    merged['expense_per_unit_month'] = np.where(
        merged['month_total_qty'] > 0,
        merged['monthly_expense'] / merged['month_total_qty'],
        np.nan)
for q in ['qty','pcs']:
    if q in merged.columns:
        merged[q] = pd.to_numeric(merged[q], errors='coerce')

# Sort by date if present
if 'date' in merged.columns:
    merged = merged.sort_values('date')


STEP 7 — Verify merge coverage

In [103]:
# Verify expense coverage
print("\nUnique year-month combinations in sales data:")
print(merged[['year','month']].drop_duplicates().sort_values(['year','month']))

print("\nUnique year-month combinations in expense data:")
print(monthly_expense[['year','month']].drop_duplicates().sort_values(['year','month']))

print("\nMonthly expense statistics:")
print(merged['monthly_expense'].describe())

zero_expense = (merged['monthly_expense'] == 0).mean() * 100
print(f"\n⚠️ {zero_expense:.2f}% of rows have monthly_expense = 0")


Unique year-month combinations in sales data:
          year  month
128989  2021.0    6.0
129708  2021.0    7.0
131077  2021.0    8.0
132018  2021.0    9.0
139220  2021.0   10.0
135034  2021.0   11.0
141634  2021.0   12.0
136485  2022.0    1.0
142742  2022.0    2.0
144421  2022.0    3.0
48246   2022.0    4.0
89793   2022.0    5.0
127869  2022.0    6.0

Unique year-month combinations in expense data:
   year  month
0  2022      6

Monthly expense statistics:
count    177008.000000
mean       1064.838877
std        2047.031076
min           0.000000
25%           0.000000
50%           0.000000
75%           0.000000
max        5000.000000
Name: monthly_expense, dtype: float64

⚠️ 78.70% of rows have monthly_expense = 0


STEP 8 — Quick sanity checks

In [104]:
print("Rows:", len(merged))
if 'monthly_expense' in merged.columns:
    print("Monthly expense stats:\n", merged['monthly_expense'].describe())
print("Sample columns:\n", list(merged.columns)[:30])

Rows: 177008
Monthly expense stats:
 count    177008.000000
mean       1064.838877
std        2047.031076
min           0.000000
25%           0.000000
50%           0.000000
75%           0.000000
max        5000.000000
Name: monthly_expense, dtype: float64
Sample columns:
 ['index', 'order_id', 'date', 'status', 'fulfilment', 'sales_channel', 'ship-service-level', 'style', 'sku', 'category', 'size', 'asin', 'courier_status', 'qty', 'currency', 'amount', 'ship-city', 'ship-state', 'ship-postal-code', 'ship-country', 'promotion-ids', 'b2b', 'fulfilled-by', 'unnamed:_22', 'months', 'customer', 'pcs', 'rate', 'gross_amt', 'style_id']


STEP 9 — Free-text detection

In [105]:
obj_cols = merged.select_dtypes(include='object').columns.tolist()
free_text_cols, structured_text_cols = [], []
n_rows = len(merged)

for col in obj_cols:
    avg_len = merged[col].astype(str).str.len().mean()
    uniq_ratio = merged[col].nunique(dropna=True) / max(n_rows, 1)
    # Heuristics: long strings or highly unique → free text
    if (avg_len >= 25) or (uniq_ratio >= 0.50):
        free_text_cols.append(col)
    else:
        structured_text_cols.append(col)

print("\n🔎 Free-text candidates:", free_text_cols)

# Save free-text subset for later NLP feature engineering (do not drop from merged unless you want)
if free_text_cols:
    merged[free_text_cols].to_pickle('/content/free_text_columns.pkl')
    print("💾 Saved free-text columns to /content/free_text_columns.pkl")



🔎 Free-text candidates: ['order_id', 'promotion-ids']
💾 Saved free-text columns to /content/free_text_columns.pkl


check missing values before eda

In [106]:
# ✅ STEP 0 — Detect dataframe automatically
df = pd.read_csv("/content/amazon_with_expense_clean.csv")

# ✅ STEP 1 — Normalize all missing values (different cases & formats)
# Converts any version of 'nan', 'null', 'none', etc. into real NaN
df = df.replace(
    to_replace=[
        r'^\s*$',              # empty strings
        r'(?i)^nan$',          # 'nan', 'NaN', 'NAN'
        r'(?i)^null$',         # 'null', 'NULL'
        r'(?i)^none$',         # 'none', 'None'
        r'(?i)^n/a$',          # 'n/a', 'N/A'
        r'(?i)^missing$',      # 'missing', 'MISSING'
    ],
    value=np.nan,
    regex=True
)

print("\n🔎 Checking missing (NaN / null / none) values before EDA...\n")

# ✅ STEP 2 — Calculate % of missing values per column
missing = df.isna().mean() * 100
missing = missing.sort_values(ascending=False)

print("📊 Top 15 columns with missing values (%):")
print(missing.head(15))

# ✅ STEP 3 — Drop columns ≥90% missing
too_empty = missing[missing >= 90].index.tolist()
if len(too_empty) > 0:
    df.drop(columns=too_empty, inplace=True)
    print(f"\n🧹 Dropped {len(too_empty)} columns (≥90% missing): {too_empty}")
else:
    print("\n🧹 No columns had ≥90% missing values.")

# ✅ STEP 4 — Fill expense-related columns with 0
for col in ['monthly_expense', 'month_total_qty', 'expense_per_unit_month']:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0)

# ✅ STEP 5 — Global missing count
total_missing = int(df.isna().sum().sum())
missing_percent_total = (total_missing / df.size) * 100

print(f"\n✅ Remaining missing values: {total_missing:,} "
      f"({missing_percent_total:.2f}% of all data points)")



🔎 Checking missing (NaN / null / none) values before EDA...

📊 Top 15 columns with missing values (%):
pcs               100.000000
tp                 99.248622
final_mrp_old      99.248622
ajio_mrp           99.248622
style_id           99.248622
mrp_old            99.248622
catalog            99.248622
flipkart_mrp       99.248622
myntra_mrp         99.248622
paytm_mrp          99.248622
snapdeal_mrp       99.248622
limeroad_mrp       99.248622
amazon_fba_mrp     99.248622
amazon_mrp         99.248622
weight             99.248622
dtype: float64

🧹 Dropped 18 columns (≥90% missing): ['pcs', 'tp', 'final_mrp_old', 'ajio_mrp', 'style_id', 'mrp_old', 'catalog', 'flipkart_mrp', 'myntra_mrp', 'paytm_mrp', 'snapdeal_mrp', 'limeroad_mrp', 'amazon_fba_mrp', 'amazon_mrp', 'weight', 'sku_code', 'color', 'design_no.']

✅ Remaining missing values: 1,547,882 (24.29% of all data points)


In [110]:
# ✅ STEP — Quantify remaining true NaN (missing) values
print("\n📉 Summary of true missing (NaN) values after cleaning:\n")

missing_summary = (
    df.isna().sum()
      .to_frame(name="Missing_Count")
      .assign(Missing_Percent=lambda x: (x["Missing_Count"] / len(df) * 100).round(2))
      .query("Missing_Count > 0")  # show only columns that have missing
      .sort_values("Missing_Percent", ascending=False)
)

if missing_summary.empty:
    print("✅ No missing values left in the dataset.")
else:
    display(missing_summary.head(15))
    print(f"\n🔢 Total columns with missing values: {missing_summary.shape[0]}")

df.to_csv("/content/amazon_clean_final.csv", index=False)
print("💾 Final clean dataset saved with latest updates!")



📉 Summary of true missing (NaN) values after cleaning:



,Missing_Count,Missing_Percent
rate,140616,79.44
customer,140616,79.44
gross_amt,140616,79.44
months,139601,78.87
fulfilled-by,137731,77.81
promotion-ids,97186,54.90
unnamed:_22,97083,54.85
currency,55828,31.54
courier_status,54905,31.02
ship-city,48068,27.16



🔢 Total columns with missing values: 23
💾 Final clean dataset saved with latest updates!


check for excluded files (cloud and p&l)

In [109]:
import pandas as pd

# Cloud file: show that it’s a provider comparison table, not sales data
try:
    cloud = pd.read_csv('/content/Cloud Warehouse Compersion Chart.csv')
    print("Cloud shape:", cloud.shape)
    print("Cloud columns:", cloud.columns.tolist())
    print("\nCloud head (3 rows):\n", cloud.head(3).to_string(index=False))
except Exception as e:
    print("Cloud file not found or unreadable:", e)

#  P&L file: quick SKU mismatch evidence
try:
    pl = pd.read_csv('/content/P  L March 2021.csv')
    print("\nP&L shape:", pl.shape)
    print("P&L columns (first 12):", list(pl.columns)[:12])

    # overlap with merged files:
    if 'merged' in globals() and 'sku' in merged.columns and 'sku' in pl.columns:
        sales_skus = set(merged['sku'].astype(str).str.strip().str.lower().dropna().unique())
        pl_skus    = set(pl['sku'].astype(str).str.strip().str.lower().dropna().unique())
        overlap = len(sales_skus & pl_skus)
        print(f"SKU overlap (sales vs P&L): {overlap} common SKUs "
              f"out of {len(sales_skus)} (sales) and {len(pl_skus)} (P&L)")
    else:
        print("SKU overlap not computed (missing 'merged' or 'sku' column).")
except Exception as e:
    print("P&L file not found or unreadable:", e)

# Clean
for _v in ['cloud','pl','sales_skus','pl_skus','overlap']:
    if _v in globals(): del globals()[_v]
print("\n check completed (no changes to the modeling dataset).")


Cloud shape: (50, 4)
Cloud columns: ['index', 'Shiprocket', 'Unnamed: 1', 'INCREFF']

Cloud head (3 rows):
  index                    Shiprocket       Unnamed: 1          INCREFF
     0                         Heads Price (Per Unit) Price (Per Unit)
     1 Inbound (Fresh Stock and RTO)            ₹4.00                4
     2                      Outbound            ₹7.00               11

P&L shape: (1330, 18)
P&L columns (first 12): ['index', 'Sku', 'Style Id', 'Catalog', 'Category', 'Weight', 'TP 1', 'TP 2', 'MRP Old', 'Final MRP Old', 'Ajio MRP', 'Amazon MRP']
SKU overlap not computed (missing 'merged' or 'sku' column).

 check completed (no changes to the modeling dataset).
